### Import Required Libraries and Set Up Environment Variables

In [5]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [6]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [9]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={os.getenv('NYT_API_KEY')}"
    f"&fq={requests.utils.quote(filter_query)}"
    f"&sort={sort}&fl={field_list}"
    f"&begin_date={begin_date}"
    f"&end_date={end_date}"
)

In [14]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    page_query_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Page {page} retrieved successfully")
    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results found on page {page}")
        break

print(f"Total reviews retrieved: {len(reviews_list)}")


Page 0 retrieved successfully
Page 1 retrieved successfully
Page 2 retrieved successfully
Page 3 retrieved successfully
Page 4 retrieved successfully
Page 5 retrieved successfully
Page 6 retrieved successfully
Page 7 retrieved successfully
Page 8 retrieved successfully
Page 9 retrieved successfully
Page 10 retrieved successfully
Page 11 retrieved successfully
Page 12 retrieved successfully
Page 13 retrieved successfully
Page 14 retrieved successfully
Page 15 retrieved successfully
Page 16 retrieved successfully
Page 17 retrieved successfully
Page 18 retrieved successfully
Page 19 retrieved successfully
Total reviews retrieved: 200


In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2017/05/11/movies/hounds-of-love-review.html",
                "snippet": "This feature debut of the Australian filmmaker Ben Young resembles a real-life couple\u2019s killing spree in the 1980s.",
                "source": "The New York Times",
                "headline": {
                    "main": "Review: \u2018Hounds of Love\u2019 Is Tense and Deadly Down Under",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "Hounds of Love",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                       

In [17]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [24]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
def extract_title(headline):
    start = headline.find('\u2018') + 1  # Find the opening quote and move one character forward
    end = headline.find('\u2019 Review')  # Find the closing quote followed by " Review"
    if start != -1 and end != -1:
        return headline[start:end]
    else:
        return headline  # Return the original headline if quotes aren't found

# Apply the function to create the new 'title' column
reviews_df['title'] = reviews_df['headline.main'].apply(extract_title)

# Display the first few rows to verify
reviews_df[['headline.main', 'title']].head()


,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,Review: ‘What’s Love Got to Do With It?’ Proba...
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist’s Guide to Love
4,‘Other People’s Children’ Review: True Romance,Other People’s Children


In [27]:
def extract_keywords(keyword_list):
    # If the input is already a string, return it as is
    if isinstance(keyword_list, str):
        return keyword_list
    
    extracted_keywords = ""
    # Check if the input is a list (of dictionaries)
    if isinstance(keyword_list, list):
        for item in keyword_list:
            # Check if the item is a dictionary
            if isinstance(item, dict):
                # Extract 'name' and 'value'
                keyword = f"{item.get('name', '')}: {item.get('value', '')}; "
                # Append the keyword item to the extracted_keywords string
                extracted_keywords += keyword
            else:
                # If it's not a dictionary, just convert it to string
                extracted_keywords += str(item) + "; "
    else:
        # If it's neither a string nor a list, convert to string
        return str(keyword_list)
    
    return extracted_keywords.strip()

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df['keywords'].head()


0    subject: Movies;creative_works: The Attachment...
1    subject: Movies;persons: Kapur, Shekhar;person...
2    subject: Movies;creative_works: You Can Live F...
3    subject: Movies;creative_works: A Tourist's Gu...
4    subject: Movies;persons: Zlotowski, Rebecca;cr...
Name: keywords, dtype: object

In [32]:
# Create a list from the "title" column using to_list()
titles = reviews_df['title'].to_list()

# Print the first few titles to verify
reviews_df['title'].head(5)

0                               The Attachment Diaries
1    Review: ‘What’s Love Got to Do With It?’ Proba...
2                                 You Can Live Forever
3                            A Tourist’s Guide to Love
4                              Other People’s Children
Name: title, dtype: object

### Access The Movie Database API

In [33]:
# Prepare The Movie Database query
base_url = "https://api.themoviedb.org/3/search/movie"
tmdb_api_key = os.getenv('TMDB_API_KEY')


movie_query_url = f"{base_url}?api_key={tmdb_api_key}&query="

In [34]:
# Prepare The Movie Database query
base_url = "https://api.themoviedb.org/3/search/movie"
tmdb_api_key = os.getenv('TMDB_API_KEY')
movie_query_url = f"{base_url}?api_key={tmdb_api_key}&query="

# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Constants for rate limiting
REQUESTS_PER_SLEEP = 50
SLEEP_TIME = 1  # seconds

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % REQUESTS_PER_SLEEP == 0:
        print(f"Sleeping for {SLEEP_TIME} seconds after {REQUESTS_PER_SLEEP} requests...")
        time.sleep(SLEEP_TIME)
    
    # Perform a "GET" request for The Movie Database
    search_url = f"{movie_query_url}{requests.utils.quote(title)}"
    response = requests.get(search_url)
    movies = response.json()

    try:
        # Check if any movies were found
        if movies['results']:
            # Get movie id
            movie_id = movies['results'][0]['id']

            # Make a request for the full movie details
            movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

            # Execute "GET" request with url
            details_response = requests.get(movie_details_url)
            movie_details = details_response.json()

            # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_details['genres']]

            # Extract the spoken_languages' English name into a list
            spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]

            # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in movie_details['production_countries']]

            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            movie_data = {
                'title': movie_details['title'],
                'original_title': movie_details['original_title'],
                'budget': movie_details['budget'],
                'original_language': movie_details['original_language'],
                'homepage': movie_details['homepage'],
                'overview': movie_details['overview'],
                'popularity': movie_details['popularity'],
                'runtime': movie_details['runtime'],
                'revenue': movie_details['revenue'],
                'release_date': movie_details['release_date'],
                'vote_average': movie_details['vote_average'],
                'vote_count': movie_details['vote_count'],
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries
            }
            tmdb_movies_list.append(movie_data)

            # Print out the title that was found
            print(f"Found movie: {movie_data['title']}")
        else:
            raise Exception("No movies found")

    except Exception as e:
        # Use the except clause to print out a statement if a movie is not found.
        print(f"Movie not found: {title}. Error: {str(e)}")

    # Increment the counter after the request (whether it succeeded or failed)
    request_counter += 1

Found movie: The Attachment Diaries
Movie not found: Review: ‘What’s Love Got to Do With It?’ Probably a Lot. Error: No movies found
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Found movie: A Thousand and One
Found movie: Your Place or Mine
Found movie: Love in the Time of Fentanyl
Found movie: Pamela, A Love Story
Found movie: In from the Side
Found movie: After Love
Found movie: Alcarràs
Found movie: Nelly & Nadine
Found movie: Lady Chatterley's Lover
Found movie: The Sound of Christmas
Found movie: The Inspection
Found movie: Bones and All
Found movie: My Policeman
Found movie: About Fate
Found movie: Waiting for Bojangles
Found movie: I Love My Dad
Found movie: A Love Song
Found movie: Alone Together
Found movie: Art of Love
Found movie: 2099: The Soldier Protocol
Found movie: Thor: Love and Thunder
Found movie: Both Sides of the Blade
Found movi

In [35]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.636,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [36]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmbd_df

NameError: name 'tmbd_df' is not defined

### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Spanish],,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[French, English]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,[English],,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[],,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,[Drama],en,[English],https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,In Search of Fellini,In Search of Fellini,0,"[Drama, Adventure]",en,[English],,"Lucy, a small-town girl from Ohio, discovers t...",4.103,93,...,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,"[{'firstname': 'Monica', 'middlename': None, '...",None
121,The Last Face,The Last Face,0,[Drama],en,"[English, Portuguese]",,"Miguel, a heroic Spanish doctor, puts himself ...",9.081,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
122,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[French],,Fiona visits Paris for the first time to assis...,6.798,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
123,The Other Half,The Other Half,0,"[Drama, Comedy, Romance]",en,[English],,When English Mark surprises his American bride...,1.058,94,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [16]:
# Drop "byline.person" column


In [17]:
# Delete duplicate rows and reset index


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [18]:
# Export data to CSV without the index
